<a href="https://colab.research.google.com/github/potatopootie/ICT2206Assignment1/blob/main/3204_STUPID_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import re
import seaborn as sns 


To input dataset and check if the columns and output is correct

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
full_df = pd.read_csv(r"shuffled-noIPV6.csv")
# df = pd.read_csv("truncated_training.csv")

print(f"[*] Shape of dataset: {full_df.shape}")
from sklearn.utils import shuffle

# Drop the first column (unnamed)
full_df.drop(full_df.columns[0], axis=1, inplace=True)
full_df = shuffle(full_df)

print(full_df.tail())
print("\n")

[*] Shape of dataset: (98274, 15)
         @timestamp           src_ip src_port         dst_ip dst_port  \
16422  1.666582e+09  192.168.207.142        0              0        0   
94503  1.666591e+09                0        0      10.0.0.21     3306   
48048  1.666692e+09     100.64.19.22     8443  100.64.19.103    43524   
59323  1.666692e+09                0        0              0        0   
50954  1.666692e+09    100.64.19.103    44618   100.64.19.22     8443   

      Protocol http_request_method http_request_referrer http_response_code  \
16422    HTTPS                 GET                     0                404   
94503        0                   0                     0                  0   
48048      TCP                   0                     0                  0   
59323        0                   0                     0                  0   
50954      TCP                   0                     0                  0   

           url_path user_agent_original sql_method s

In [ ]:
# Doing further cleaning 
full_df["src_port"]=  full_df["src_port"].replace(regex=",", value= "")
full_df["src_port"]=  full_df["src_port"].replace(regex=",", value= "")
full_df["dst_port"]=  full_df["dst_port"].replace(regex=",", value= "")
full_df["dst_port"]=  full_df["dst_port"].replace(regex=" ", value= "")
full_df["src_port"] = full_df["src_port"].replace(regex=" ", value="")
full_df["src_port"] = full_df["src_port"].replace(regex="dns", value="53")
full_df["dst_port"] = full_df["dst_port"].replace(regex="dns", value="53")
full_df["dst_ip"] = full_df["dst_ip"].replace(regex="\S*:+\S+", value="0")

In [ ]:
# metrics module implements functions assessing prediction error for specific purposes 
from sklearn import metrics
scores = {}
scores_list= []
# function for results of the algo
def scoring_metrics(y_test, y_pred, model): 
    print(f"y_test size:{y_test.size} y_pred size:{y_pred.size}")
    accuracy = metrics.accuracy_score(y_test, y_pred)
    scores[model] = accuracy
    scores_list.append(accuracy)
#     confusion_matrix = metrics.confusion_matrix(y_true=[True, True], y_pred=[True, True], labels=[True, False])
    cm = confusion_matrix(y_test, y_pred, labels=['nmap_scan', 'port_scan', 'smtp_enumeration', 'sql_enumeration', 'web_enumeration'])
    print(f"Confusion Matrix: {cm}")

    print(f"\n[*] Model: {model}")
    print("[*] Precision: {:.3f}%".format(metrics.precision_score(y_test, y_pred, average="weighted")))
    print("[*] Recall: {:.3f}%".format(metrics.recall_score(y_test, y_pred, average="weighted")))

    print("[*] Accuracy: {:.3f}%".format(accuracy))
    print("[*] F1_score: {:.3f}%".format(metrics.f1_score(y_test, y_pred, average="weighted")))

In [ ]:
df = full_df.head(15000)
# df = full_df

In [ ]:
print("Doing")
from sklearn.preprocessing import OneHotEncoder
df.replace(to_replace=["None"], value=np.nan, inplace=True)
clean_df = df.fillna(str(0)) #uhm i dont knowhow to explain this, but please lmk another alternative to fix this cause :") try to run it without str and you will know what i mean"
clean_x = clean_df.iloc[:, :13].values
clean_y = clean_df["category"].values
features = df.columns.values[:-1]

for label in clean_df.columns:
    for index, rows in clean_df.iterrows():
        new_ip = ""
        ip = str(rows[label])
        if re.search("\d+\.\d+\.\d+\.\d+", ip):
            octets = ip.split(".")
            for octet in octets:
                octet = octet.rjust(3,"0")
                new_ip += octet
            clean_df[label][index] = new_ip

clean_df["http_response_code"] = clean_df["http_response_code"].replace('HTTP/1.1"', value="0")
clean_df["src_ip"] = clean_df["src_ip"].replace('::1', value="0")
clean_df["dst_ip"] = clean_df["dst_ip"].replace('::1', value="0")
# clean_df["http_response_code"] = clean_df["http_response_code"].replace('HTTP/1.1"', value="0")
print("Done")

Doing


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Done


In [ ]:
from sklearn.compose import make_column_transformer
clean_x = clean_df.iloc[:, :13] #honestly don't know if this is correct lmaoooo (To take all the columns except for the last one)
#this is to iterate through the columns and convert the strings to float (passthrough means for those columns we didnt specify, leave it as it is)
column_trans = make_column_transformer((OneHotEncoder(sparse=False), ['Protocol', 'http_request_method', 'http_request_referrer', 'url_path', 'user_agent_original', 'sql_method', 'sql_query']),remainder='passthrough')
test = column_trans.fit_transform(clean_x) #this is technically our cl

In [ ]:
from sklearn.model_selection import train_test_split
#clean_y is category 
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
x_train, x_test, y_train, y_test = train_test_split(test, clean_y, test_size=0.3)

Train the algorithm

FOR RANDOM FOREST CLASSIFIER 
criterion='gini' = measure the quality of the split 

max_depth = maximum depth of the tree, if none, then nodes are expanded until all trees are pure (?) 

min_samples_split = minimum number of samples required to split an internal node

random_state = controls both the randomness and bootstrapping of the samples when building trees 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(criterion="gini",
                             max_depth= 10,
                             min_samples_split = 10,
                             random_state=5) #can put any number here

In [ ]:
clf.fit(x_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_split=10, random_state=5)

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
y_pred

array(['port_scan', 'port_scan', 'port_scan', ..., 'port_scan',
       'port_scan', 'port_scan'], dtype=object)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 390,    1,    0,    0,    0],
       [   0, 2256,    0,    0,    0],
       [   0,    8,    0,    0,    0],
       [   0,  210,    0,  328,    0],
       [   0,  180,    0,    0, 1127]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9113333333333333

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf, x_train, y_train, cv=10)

array([0.91333333, 0.94190476, 0.8247619 , 0.83047619, 0.90285714,
       0.90761905, 0.91809524, 0.92095238, 0.94380952, 0.90761905])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

                  precision    recall  f1-score   support

       nmap_scan       1.00      1.00      1.00       390
       port_scan       1.00      0.85      0.92      2655
smtp_enumeration       0.00      0.00      0.00         0
 sql_enumeration       0.61      1.00      0.76       328
 web_enumeration       0.86      1.00      0.93      1127

        accuracy                           0.91      4500
       macro avg       0.69      0.77      0.72      4500
    weighted avg       0.94      0.91      0.92      4500



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
